In [ ]:
import cv2
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from keras.models import load_model
import numpy as np

faceCascade = cv2.CascadeClassifier('J:/11th Semester/CVPR/FINAL/Mask_Detection/SavedFiles/haarcascade_frontalface_alt2.xml')

model_path = 'J:/11th Semester/CVPR/FINAL/Mask_Detection/SavedFiles/model.hdf5'
model = load_model(model_path)

def face_mask_detector(frame):
  # frame = cv2.imread(fileName)
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = faceCascade.detectMultiScale(gray,
                                        scaleFactor=1.1,
                                        minNeighbors=5,
                                        minSize=(60, 60),
                                        flags=cv2.CASCADE_SCALE_IMAGE)

  faces_list=[]
  preds= []
  for (x, y, w, h) in faces:
      face_frame = frame[y:y+h,x:x+w]
      face_frame = cv2.cvtColor(face_frame, cv2.COLOR_BGR2RGB)
      face_frame = cv2.resize(face_frame, (224, 224))
      face_frame = img_to_array(face_frame)
      face_frame = np.expand_dims(face_frame, axis=0)
      face_frame =  preprocess_input(face_frame)
      faces_list=[]
      faces_list.append(face_frame)
      if len(faces_list)>0:
          preds = model.predict(faces_list)
      for pred in preds:
          (mask, withoutMask) = pred
      label = "Mask" if mask > withoutMask else "No Mask"
      color = (0, 255, 0) if label == "Mask" else (0, 0, 255)
      label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
      cv2.putText(frame, label, (x, y- 10),
                  cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

      cv2.rectangle(frame, (x, y), (x + w, y + h),color, 3)
  # cv2_imshow(frame)
  return frame

output_video_path = 'J:/11th Semester/CVPR/FINAL/Mask_Detection/VideoOutput/output.avi'
cap = cv2.VideoCapture('J:/11th Semester/CVPR/FINAL/Mask_Detection/VideoFile/Wearing_a_mask.mp4')
ret, frame = cap.read()
frame_height, frame_width, _ = frame.shape

# Create the VideoWriter object with the specified output path
out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width, frame_height))

print("Processing Video...")
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        out.release()
        break
    output = face_mask_detector(frame)
    out.write(output)

out.release()
print("Done processing video")

Processing Video...
